# Εισαγωγή scikit-learn και dataset στο notebook

Αρχικά κάνουμε upgrade στις βιβλοθήκες που θα χρειαστούμε, αν δεν έχουμε ήδη κάνει. Στο Colab θα χρεαστεί πιθανότατα να κάνουμε restart το runtime.

In [1]:
!pip install --upgrade pip
!pip install scikit-learn --upgrade 
!pip install numpy --upgrade 
!pip install --upgrade matplotlib 
!pip install --upgrade pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

![UCI ML Logo](http://archive.ics.uci.edu/ml/assets/logo.gif "UCI Machine Learning Repository")

To [UCI ML Repository](http://archive.ics.uci.edu/ml/index.php) είναι το διασημότερο αποθετήριο datasets για Machine Learning. Το dataset με το οποίο θα δουλέψουμε είναι το [Dermatology Database](http://archive.ics.uci.edu/ml/datasets/Dermatology). Το dataset περιλαμβάνει διάφορες πληροφορίες σχετικά με δερματολογικές παθήσεις, συγκεκριμένα ESD (Erythemato-Squamous Diseases) καθώς και ετικέτες για κάθε δείγμα (sample), αν αυτό αντιστοιχεί σε μία από τις έξι κατηγορίες ESD. Το σύνολο δεδομένων έχει 365 δείγματα και περιλαμβάνει 34 χαρακτηριστικά (attributes) για κάθε δείγμα, είτε κλινικά  π.χ. βαθμός εξανθήματος, φαγούρα, οικογενειακό ιστορικό είτε ιστοπαθολογικά όπως άκράτεια μελανίνης, εξωκυττάρωση, παρουσία ηωσινόφιλων. Θα χρησιμοποιήσουμε αυτό το dataset και τα χαρακτηριστικά για να προβλέψουμε σε ποια κατηγορία δερματολογικής πάθησης υπάγεται κάθε δείγμα.


In [2]:
import pandas as pd
import numpy as np

def conv(val):
    try:
        return float(val)
    except:
        return np.nan

# Read CSV and replace the uknown Age values with the average Age.
dataset = pd.read_csv("codon_usage.csv", low_memory=False, converters={'UUU': conv, 'UUC':conv})
print(dataset.shape)

print(dataset.isnull().sum().sort_values(ascending=False))



(13028, 69)
UUU        2
UUC        1
Kingdom    0
UAU        0
UGU        0
          ..
GGU        0
GGC        0
GGA        0
GGG        0
UGA        0
Length: 69, dtype: int64


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13028 entries, 0 to 13027
Data columns (total 69 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Kingdom      13028 non-null  object 
 1   DNAtype      13028 non-null  int64  
 2   SpeciesID    13028 non-null  int64  
 3   Ncodons      13028 non-null  int64  
 4   SpeciesName  13028 non-null  object 
 5   UUU          13026 non-null  float64
 6   UUC          13027 non-null  float64
 7   UUA          13028 non-null  float64
 8   UUG          13028 non-null  float64
 9   CUU          13028 non-null  float64
 10  CUC          13028 non-null  float64
 11  CUA          13028 non-null  float64
 12  CUG          13028 non-null  float64
 13  AUU          13028 non-null  float64
 14  AUC          13028 non-null  float64
 15  AUA          13028 non-null  float64
 16  AUG          13028 non-null  float64
 17  GUU          13028 non-null  float64
 18  GUC          13028 non-null  float64
 19  GUA 

Η στήλη Species name δεν μπορεί να χρησιμοποιηθεί, καθώς είναι τύπου string. Μπορούμε να την αφαιρέσουμε, αφού το SpeciesID μας είναι αρκετό για τον καθορισμό του είδους

In [4]:
# Cannot process string values in MLP. 
# We have the ID as index se we can drop the whole column
dataset = dataset.drop(columns='SpeciesName')

 To dataset έχει 3 ΝaN τιμές. Αυτές πρέπει να αλλάξουν για να μπορέσουμε να συνεχίσουμε.

In [5]:
# Drop the Null Values. These are 2 out of 13028 rows
dataset.dropna(inplace=True)
dataset.set_index("SpeciesID")

print("The NaN values now are: " + str(dataset.isnull().sum().sum()))
print(dataset.shape)

The NaN values now are: 0
(13026, 68)


In [6]:
# Extract the values
label_names, counts = np.unique(dataset["Kingdom"], return_counts=True) 
labels = dataset["Kingdom"].values
features = dataset.iloc[:,1:].values
feature_names = dataset.columns[1:]

In [7]:
unique, counts = np.unique(labels, return_counts=True)

for u, c in zip(unique, counts):
    print(str(u)+':'+str(c))

arc:126
bct:2919
inv:1345
mam:572
phg:220
plm:18
pln:2523
pri:180
rod:215
vrl:2831
vrt:2077


To dataset δεν είναι ισορροπημένο. Το plm εμφανίζεται 18 φορές, ενώ το bct 2919.
Περισσότερες από 100 φορές παραπάνω.

In [8]:
# οι διαστάσεις όλων των χαρακτηριστικών
print(features.shape)


(13026, 67)


In [9]:
from sklearn.model_selection import train_test_split

# Split our data
train, test, train_labels, test_labels = train_test_split(features, labels, test_size=0.3)

In [55]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score
import time
import warnings
warnings.filterwarnings('ignore')

kaggle_accuracy = {}
kaggle_f1 = {}
kaggle_times = {}

def dummy_metrics(solver, const = None):

    name = 'dummy_' + solver
    model = DummyClassifier(strategy=solver, constant=const)
    start = time.time()
    model.fit(train, train_labels)
    fit_time  = time.time() - start
    preds = model.predict(test)

    kaggle_times[name] = fit_time
    kaggle_accuracy[name] = accuracy_score(test_labels, preds)
    kaggle_accuracy[name] = f1_score(test_labels, preds, average='macro')

dummy_metrics('uniform')
dummy_metrics('most_frequent')
dummy_metrics('stratified')
dummy_metrics('constant', 'arc')
dummy_metrics('constant', 'mam')

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
import time
import warnings
warnings.filterwarnings('ignore')

kaggle_accuracy = {}
kaggle_f1 = {}
kaggle_times = {}

def clf_metrics(solver):

    name = 'clf_' + solver
    model = MLPClassifier(solver=solver)
    start = time.time()
    model.fit(train, train_labels)
    fit_time  = time.time() - start
    preds = model.predict(test)

    kaggle_times[name] = fit_time
    kaggle_accuracy[name] = accuracy_score(test_labels, preds)
    kaggle_accuracy[name] = f1_score(test_labels, preds, average='macro')

clf_metrics('adam')
clf_metrics('lbfgs')
clf_metrics('sgd')



## Σχόλιο

Ο SVM, αργεί πάρα πολύ να κάνει fit, αν δεν μπει όριο στο max_iter. Αυτό είναι λογικό, αφού κρατά έναν ΝxN πίνακα για τις αποστάσεις όλων των σημείων. Με 13000 εγγραφές, καταλαβαίνουμε πως αυτός ο πίνακας είναι πολύ μεγάλος για να γίνει γρήγορα η επεξεργασία του. Για τώρα, θα περιορίσουμε τις επαναλήψεις του, ενώ, κατά την βελτιστοποίηση, θα μικρύνουμε αρκετά το dataset μας.

In [50]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
import time
import warnings
warnings.filterwarnings('ignore')

def svc_metrics(solver):

    name = 'svc_' + solver
    model = SVC(kernel=solver, random_state=0, C=1, max_iter=100000)
    start = time.time()
    model.fit(train, train_labels)
    fit_time  = time.time() - start
    preds = model.predict(test)

    kaggle_times[name] = fit_time
    kaggle_accuracy[name] = accuracy_score(test_labels, preds)
    kaggle_accuracy[name] = f1_score(test_labels, preds, average='macro')

svc_metrics('linear')
svc_metrics('poly')
svc_metrics('rbf')

In [56]:
print(kaggle_times)

{'dummy_uniform': 0.013653993606567383, 'dummy_most_frequent': 0.0030519962310791016, 'dummy_stratified': 0.0023932456970214844, 'dummy_constant': 0.002031087875366211}
